In [1]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import csv
import numpy as np

In [6]:
directors = []
actors = []
mpaaRatings = []
X = [] # This holds the movie data that the model will use to train.
y = [] # This holds whether the movies won an oscar or not that the model will use to train.

dummyData = pd.read_csv("../Data/oscars.csv", encoding = "ISO-8859-1")
n = len(dummyData.index)

for i in range(0, n):
    movie = list(dummyData.iloc[i])
    director = movie[7]
    actor1 = movie[11]
    actor2 = movie[15]
    actor3 = movie[19]
    mpaaRating = movie[23]
    
    if not(director in directors):
        directors.append(director)
    movie[7] = directors.index(director) # replace director to its unique numerical value
    movie[8] = int(movie[8])
    
    if not(actor1 in actors):
        actors.append(actor1)
    movie[11] = actors.index(actor1)
    movie[12] = int(movie[12])
    
    if not(actor2 in actors):
        actors.append(actor2)
    movie[15] = actors.index(actor2)
    movie[16] = int(movie[16])
    print(i , ": " , movie[16])
    
    if not(actor3 in actors):
        actors.append(actor3)
    movie[19] = actors.index(actor3)
    movie[20] = int(movie[20])
    
    if not(mpaaRating in mpaaRatings):
        mpaaRatings.append(mpaaRating)
    movie[23] = mpaaRatings.index(mpaaRating) # replace mpaa rating to its unique numerical value
    
    X.append(movie[3:])
    y.append(int(movie[1]))

0 :  0
1 :  0
2 :  0
3 :  0
4 :  0
5 :  0
6 :  0
7 :  0
8 :  0
9 :  0
10 :  0
11 :  0
12 :  0
13 :  0
14 :  0
15 :  0
16 :  0
17 :  0
18 :  0
19 :  0
20 :  0
21 :  1
22 :  0
23 :  0
24 :  0
25 :  0
26 :  0
27 :  0
28 :  0
29 :  0
30 :  0
31 :  0
32 :  0
33 :  0
34 :  0
35 :  0
36 :  0
37 :  0
38 :  0
39 :  0
40 :  0
41 :  0
42 :  0
43 :  0
44 :  0
45 :  0
46 :  0
47 :  0
48 :  0
49 :  0
50 :  0
51 :  0
52 :  0
53 :  0
54 :  0
55 :  0
56 :  0
57 :  0
58 :  0
59 :  0
60 :  0
61 :  0
62 :  0
63 :  0
64 :  0
65 :  0
66 :  0
67 :  0
68 :  0
69 :  0
70 :  0
71 :  0
72 :  0
73 :  0
74 :  0
75 :  0
76 :  0
77 :  0
78 :  0
79 :  1
80 :  0
81 :  0
82 :  0
83 :  0
84 :  0
85 :  0
86 :  0
87 :  0
88 :  0
89 :  0
90 :  0
91 :  0
92 :  0
93 :  0
94 :  0
95 :  0
96 :  0
97 :  0
98 :  0
99 :  0
100 :  0
101 :  0
102 :  0
103 :  0
104 :  0
105 :  0
106 :  0
107 :  0
108 :  0
109 :  0
110 :  0
111 :  1
112 :  0
113 :  0
114 :  0
115 :  0
116 :  0
117 :  0
118 :  0
119 :  0
120 :  0
121 :  0
122 :  0
123

In [11]:
data = []
data.append(["C", "gamma", "Round 1", "Round 2", "Round 3", "Round 4", "Round 5", "Round 6", "Round 7", "Round 8", "Round 9", "Round 10", "Avg F1 Score", "Avg Accuracy"])
    
c_values = [0.1, 1, 10, 100, 1000, 1100]
gamma_values = [0.01, 0.1, 1, 10, 100]

for i in range(0,6):
    for j in range(0,5):
        data_row = []
        data_row.append(c_values[i])
        data_row.append(gamma_values[j])
        f1_scores = []
        accuracy = []
        for k in range(0, 10):
            # train_test_split takes the datasets, shuffles the datasets, and splits it into training and testing sets.
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)

            svc = make_pipeline(StandardScaler(), SVC(probability=True, class_weight='balanced', C=c_values[i], gamma=gamma_values[j]))
            svc.fit(X_train, y_train)

            predictions = svc.predict(X_test) # This predict function produces yes or no Oscar predictions.
            proba_predictions = svc.predict_proba(X_test)[:,1] # This predict_proba function produces the probability of the movies winning an oscar.
            
            f1_scores.append(f1_score(y_test, predictions, average='weighted'))
            data_row.append(round(f1_score(y_test, predictions, average='weighted'),3))
            accuracy.append(accuracy_score(y_test,predictions))
        
        avg_f1 = np.mean(f1_scores)
        avg_accuracy = np.mean(accuracy)
        data_row.append(round(avg_f1,3))  
        data_row.append(round(avg_accuracy,3))
        data.append(data_row)
with open("svc_data_parametertuning2sample.csv","w+",newline='') as csv_file:
    csvWriter = csv.writer(csv_file,delimiter=',')
    csvWriter.writerows(data)

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Raymond Vu\Anaconda3\lib\site-p

In [44]:
# This predict function produces yes or no Oscar predictions.
predictions = svc.predict(X_test)
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("Predictions\tActual")
print(pretty_list)

Predictions	Actual
0		0
0		0
0		0
0		0
0		1
0		0
0		0
1		1
0		0
0		0
0		0
1		1
1		0
0		0
0		0
0		0
0		0
1		1
1		0
0		0
0		0
1		0
0		0
0		0
1		0


In [45]:
# This predict_proba function produces the probability of the movies winning an oscar.
proba_predictions = svc.predict_proba(X_test)[:,1]
pretty_list = "\n".join("{:.2f}\t{}".format(x, y) for x, y in zip(proba_predictions, y_test))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.12	0
0.13	0
0.15	0
0.11	0
0.17	1
0.11	0
0.09	0
0.18	1
0.17	0
0.11	0
0.13	0
0.17	1
0.18	0
0.12	0
0.12	0
0.14	0
0.17	0
0.27	1
0.21	0
0.13	0
0.12	0
0.18	0
0.16	0
0.12	0
0.21	0


In [46]:
cm=confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test,predictions)

[[17  4]
 [ 1  3]]


0.8

In [10]:
C = np.logspace(0, 4, 10)
print(C)

[1.00000000e+00 2.78255940e+00 7.74263683e+00 2.15443469e+01
 5.99484250e+01 1.66810054e+02 4.64158883e+02 1.29154967e+03
 3.59381366e+03 1.00000000e+04]
